In [3]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import numpy as np


In [4]:
df_train_features = pd.read_csv("../data/train_features.csv")
sites = df_train_features["site"].unique()

In [5]:
import sys
sys.path.append("../functions")


In [6]:
example_images = df_train_features.groupby("site").value_counts()[sites[1]].to_frame().reset_index()
labels = pd.read_csv("../data/train_labels.csv")

In [7]:
merged_df = pd.merge(example_images, labels, on="id", how="left")
print(merged_df)

           id                     filepath  count  antelope_duiker  bird   
0    ZJ002485  train_features/ZJ002485.jpg      1              0.0   0.0  \
1    ZJ005399  train_features/ZJ005399.jpg      1              1.0   0.0   
2    ZJ005451  train_features/ZJ005451.jpg      1              0.0   0.0   
3    ZJ005613  train_features/ZJ005613.jpg      1              0.0   0.0   
4    ZJ005766  train_features/ZJ005766.jpg      1              0.0   0.0   
..        ...                          ...    ...              ...   ...   
212  ZJ014454  train_features/ZJ014454.jpg      1              0.0   0.0   
213  ZJ014580  train_features/ZJ014580.jpg      1              0.0   0.0   
214  ZJ014585  train_features/ZJ014585.jpg      1              0.0   0.0   
215  ZJ014645  train_features/ZJ014645.jpg      1              1.0   0.0   
216  ZJ015025  train_features/ZJ015025.jpg      1              0.0   0.0   

     blank  civet_genet  hog  leopard  monkey_prosimian  rodent  
0      0.0          0

In [8]:
example_images = df_train_features.groupby("site").value_counts()[sites[1]].to_frame().reset_index()["filepath"].to_list()

shape_list = []
for file in example_images:
    file_path = "../data/" + file
    img = cv.imread(file_path)
    shape_list.append(img.shape)

sls = pd.Series(shape_list)
sls.value_counts()

(540, 960, 3)    217
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
print(merged_df.columns.tolist())

['id', 'filepath', 'count', 'antelope_duiker', 'bird', 'blank', 'civet_genet', 'hog', 'leopard', 'monkey_prosimian', 'rodent']


In [11]:
#print(merged_df.columns.tolist())
#print(merged_df.info())
y = merged_df.drop(["filepath", "id", "count"], axis=1)
X = []
print(y.shape)
for filepath in merged_df["filepath"]:
    img = cv.imread("../data/" + filepath)
    X.append(img)
X = np.asarray(X)
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

(217, 8)
(217, 540, 960, 3)


In [12]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

model = keras.Sequential([
 Flatten(input_shape=(540, 960, 3)),
 Dense(8, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=2, batch_size = 32)

Epoch 1/2


/Users/riael/.pyenv/versions/3.11.3/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.1366 - loss: 145518.5938
Epoch 2/2
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.3267 - loss: 128478.9375


In [13]:
loss, acc = model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5286 - loss: 180185.8750


In [18]:
from sklearn.linear_model import LogisticRegression

#X_train_flat = np.reshape(X_train, (len(X_train), -1))
#X_test_flat = np.reshape(X_test, len(X_test), -1)
logistic_reg_model = LogisticRegression(solver="lbfgs", random_state=42)
logistic_reg_model.fit(X_train, y_train)
predictions = logistic_reg_model.predict(X_test)

ValueError: Found array with dim 4. LogisticRegression expected <= 2.